### Proyecto 2C - Validación de estructura de tablas

#### Definimos una tabla para almacenar la estructura (Structure) de una tabla

Esta es la estructura de la tabla que crearemos

<center><img src="https://i.postimg.cc/ydpgrxYx/adf427.png"></center>

Despues de haber creado la tabla **tbl_FileDetails** insertamos dos registros

<center><img src="https://i.postimg.cc/GmPX82D8/adf444.png"></center>
<center><img src="https://i.postimg.cc/nMb1MbcG/adf445.png"></center>

#### Definir un Stored Procedure Dinámico para leer la Estructura (Structure) de un archivo

<center><img src="https://i.postimg.cc/xTB0pHvB/adf428.png"></center>

#### Validar la estructura de dos archivos

Recordar que tenemos estos dos archivos **IPL-Players-1.csv** y **IPL-Players-2.csv** en nuestro contenedor **ipl**

<center><img src="https://i.postimg.cc/XqSmbFWq/adf429.png"></center>

Creamos un nuevo Pipeline llamado **pl_validateFiles** y agregamos una actividad **Get Metadata**

<center><img src="https://i.postimg.cc/9Q8kfs2v/adf430.png"></center>

Vamos a obtener la metadata de la opción **Structure** del **Field list**, para el archivo **IPL-Players-1.csv** utilizando
el dataset de origen **adf_ds_Fifa_source_csv**

<center><img src="https://i.postimg.cc/m2VXdKPt/adf431.png"></center>
<center><img src="https://i.postimg.cc/44QBdjbB/adf432.png"></center>

Luego de Validar y ejecutar nuestro pipeline utilizando **Debug**, revisamos el **Output**

<center><img src="https://i.postimg.cc/155GCXhZ/adf433.png"></center>

Vemos que la opción **Structure** nos devuelve los nombres de las columnas y su tipo de dato del archivo **IPL-Players-1.csv**

<center><img src="https://i.postimg.cc/Df5sXhWp/adf434.png">
        <img src="https://i.postimg.cc/bvHdf0kV/adf435.png"></center>

Ahora vamos a obtener la metadata de la opción **Structure** del **Field list**, para el archivo **IPL-Players-2.csv** utilizando
el dataset de origen **adf_ds_Fifa_source_csv**. De igual manera, el **Output** nos devuelve los nombres de las columnas y su 
tipo de dato del archivo **IPL-Players-2.csv**

<center><img src="https://i.postimg.cc/4x54yf2B/adf436.png"></center>
<center><img src="https://i.postimg.cc/G2s1Fdz0/adf437.png"></center>

Agregamos una actividad **If Condition** y establecemos su configuración

<center><img src="https://i.postimg.cc/x14TD0s7/adf438.png"></center>

Tenemos la siguiente **Expression**:
```
equals(): Devuelve "true" si dos valores son iguales

@equals(
        activity('GetStructure_S').output.structure,
        activity('GetStructure_T').output.structure        
)
```
<center><img src="https://i.postimg.cc/nzt6D5RT/adf439.png"></center>

Configuramos la opción **True**, esta se ejecuta si la **Expression** es verdadera

<center><img src="https://i.postimg.cc/Zq6Q4HbN/adf440.png"></center>

Configuramos la opción **False**, esta se ejecuta si la **Expression** es falsa

<center><img src="https://i.postimg.cc/NjHJkDrn/adf441.png"></center>
<center><img src="https://i.postimg.cc/W3XX1fQd/adf442.png"></center>

Dado que ambos archivos **IPL-Players-1.csv** y **IPL-Players-2.csv** tienen la MISMA ESTRUCTURA de columnas,
la expresión es VERDADERA.

<center><img src="https://i.postimg.cc/L6qNmDvs/adf443.png"></center>

#### Almacenar detalles de la estructura (Structure details) en una tabla SQL

Para ello utilizamos el script SQL que se encuentra en el archivo **Update-Struct.txt**. El fin de ello es
actualizar los valores de la columna **Structure** de los dos registros que tiene la tabla **tbl_FileDetails**

<center><img src="https://i.postimg.cc/MTv6pXTV/adf446.png"></center>
<center><img src="https://i.postimg.cc/1zbTDzBp/adf447.png"></center>

#### Validación de la estructura mediante Azure SQL table y Stored Procedures

Vamos a comparar la estructura (Structure) obtenida por la actividad **Get Metadata** (la metadata de la opción **Structure** del **Field list**, para el archivo **IPL-Players-1.csv** utilizando el dataset de origen **adf_ds_Fifa_source_csv**) que nos devuelve
los nombres de las columnas de dicho archivo, frente a la estructura (Structure) que se tiene como valor en la columna **Structure**
en la tabla **tbl_FileDetails**.

Para ello eliminamos la segunda actividad **Get Metadata** y utilizamos la actividad **Lookup**

<center><img src="https://i.postimg.cc/L5xSj1sx/adf449.png"></center>

La cual trabaja con el dataset de destino **adf_ds_Fifa_Sql_table** y le entregamos como valor para sus parametros el schema **dbo**
y la tabla **tbl_FileDetails**. Activamos **First row only** para que capture solo el primer registro de la tabla, y dado que la estructura existe como un valor en el primer registro en la columna **Structure** en la tabla **tbl_FileDetails**, nos facilitará
capturar dicho valor.

De igual forma añadimos nuestro Stored Procedure, que basicamente es una Query de consulta, la cual posee la variable **@FileType**
la cual nos permite indicarle el valor que tendrá la columna **FileType** y de esta manera poder filtrar el registro que necesitamos
en la tabla **tbl_FileDetails** para poder obtener el valor de la estructura que necesitamos.
```
CREATE PROCEDURE [dbo].[uspGetFileDetails]
    @FileType  [nvarchar](60)
AS  
BEGIN
    SET NOCOUNT ON;
	
select 
	ContainerName,
	[FileName],
	SchemaName,
	TableName,
	Structure,
	[P_ID],
	[P_SCD]

from [dbo].[tbl_FileDetails]
where  [FileType] = @FileType and [Active] = 1;

END
```
<center><img src="https://i.postimg.cc/LXRF7TYy/adf450.png"></center>

Aqui le indicamos el valor a la variable **@FileType** de nuestro Stored Procedure

<center><img src="https://i.postimg.cc/Pxd0K599/adf451.png"></center>

Este es el **Output** de la actividad **Lookup**. Vemos que el valor de **Structure** es algo extraño, para ello 
tendremos que realizar una modificación en la Expresión de la actividad **If Conditional**

<center><img src="https://i.postimg.cc/vThYXK0L/adf448.png"></center>

```
Esta será la expresión de la actividad "If Conditional"

@equals(
	activity('GetStructure_S').output.structure,
	json(activity('LookupFileDetails').output.firstRow.structure) <-- Anteponemos 'json' para que tome el formato de un json
)
```
<center><img src="https://i.postimg.cc/P5jGNyKd/adf453.png"></center>
<center><img src="https://i.postimg.cc/vBP2zLWx/adf452.png"></center>
